### dataloader

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset

In [50]:
import os

files = os.listdir('../data/csv_files')
files

['2024-11-22-14-49-58.140.csv',
 '2024-11-22-14-52-58.169.csv',
 '2024-11-22-14-55-15.913.csv',
 '2024-11-22-14-56-28.671.csv',
 '2024-11-22-14-59-17.969.csv',
 '2024-11-22-15-00-23.410.csv',
 '2024-11-22-15-01-31.563.csv',
 '2024-11-22-15-02-36.778.csv',
 '2024-11-22-15-04-28.957.csv',
 '2024-11-22-15-05-39.475.csv']

In [56]:
actions_list = []
total_pose = 0
for file in files:
    path = f'../data/csv_files/{file}'
    df = pd.read_csv(path)
    actions_list += list(df.action.unique())
    total_pose += len(df)
total_pose

16588

In [59]:
import numpy as np
np.unique(actions_list)

array(['ACTIVATE', 'CROUCH', 'IDLE', 'INFO_DISPLAYED', 'JUMP',
       'LEFT_STEP', 'RIGHT_STEP', 'SHAKE'], dtype='<U14')

In [61]:
actions2labels = {'ACTIVATE':1, 
 'CROUCH':2, 
 'IDLE':0, # nothing
 'INFO_DISPLAYED':0, 
 'JUMP':3,
 'LEFT_STEP':0, 
 'RIGHT_STEP':0, 
 'SHAKE': 4}

def convert_frame_csv(df):
    
    def split_tuple_column(col):
        return col.apply(lambda x: tuple(map(float, x.strip("()").split(','))))

    df_ = pd.DataFrame(df['relative_time'])
    # Transformer le DataFrame
    for col in df.columns[1:-1]:
        # Convertir les chaînes en tuples
        tuples = split_tuple_column(df[col])
        
        # Créer les nouvelles colonnes pour x, y, z, p
        df_[f'{col}_x'] = tuples.apply(lambda t: t[0])
        df_[f'{col}_y'] = tuples.apply(lambda t: t[1])
        df_[f'{col}_p'] = tuples.apply(lambda t: t[3])

    df_ = pd.concat([df_, df['action']], axis=1)

    return df_

In [ ]:
from tqdm import tqdm

for file in tqdm(files):
    df = pd.read_csv(f'../data/csv_files/{file}')
    df = convert_frame_csv(df)
    df['action_code'] = 0
    for action in df.action.unique():
        df.loc[df.action == action, 'action_code'] = actions2labels[action]
    
    df.drop(['action'], axis=1, inplace=True)
    df.to_csv(f'../data/csv_files_converted/{file}', index=False)

In [40]:
def split_tuple_column(col):
    return col.apply(lambda x: tuple(map(float, x.strip("()").split(','))))

df_ = pd.DataFrame(df['relative_time'])
# Transformer le DataFrame
for col in df.columns[1:-1]:
    # Convertir les chaînes en tuples
    tuples = split_tuple_column(df[col])
    
    # Créer les nouvelles colonnes pour x, y, z, p
    df_[f'{col}_x'] = tuples.apply(lambda t: t[0])
    df_[f'{col}_y'] = tuples.apply(lambda t: t[1])
    df_[f'{col}_p'] = tuples.apply(lambda t: t[3])

df_ = pd.concat([df_, df['action']], axis=1)

df_

,relative_time,NOSE_x,NOSE_y,NOSE_p,LEFT_EYE_INNER_x,LEFT_EYE_INNER_y,LEFT_EYE_INNER_p,LEFT_EYE_x,LEFT_EYE_y,LEFT_EYE_p,...,RIGHT_HEEL_x,RIGHT_HEEL_y,RIGHT_HEEL_p,LEFT_FOOT_INDEX_x,LEFT_FOOT_INDEX_y,LEFT_FOOT_INDEX_p,RIGHT_FOOT_INDEX_x,RIGHT_FOOT_INDEX_y,RIGHT_FOOT_INDEX_p,action
0,0.000003,0.908151,0.377057,0.991300,0.907933,0.337525,0.989030,0.910916,0.334304,0.989894,...,0.917690,1.573847,0.106442,0.855478,1.598760,0.189950,0.826515,1.628760,0.062048,LEFT_STEP
1,0.000185,0.908151,0.377057,0.991300,0.907933,0.337525,0.989030,0.910916,0.334304,0.989894,...,0.917690,1.573847,0.106442,0.855478,1.598760,0.189950,0.826515,1.628760,0.062048,LEFT_STEP
2,0.000274,0.878059,0.377101,0.992086,0.881818,0.339911,0.990027,0.885482,0.337215,0.990794,...,0.851452,1.302500,0.113600,0.757849,1.355196,0.202173,0.767846,1.290459,0.065722,LEFT_STEP
3,0.056491,0.878059,0.377101,0.992086,0.881818,0.339911,0.990027,0.885482,0.337215,0.990794,...,0.851452,1.302500,0.113600,0.757849,1.355196,0.202173,0.767846,1.290459,0.065722,LEFT_STEP
4,0.056658,0.878059,0.377101,0.992086,0.881818,0.339911,0.990027,0.885482,0.337215,0.990794,...,0.851452,1.302500,0.113600,0.757849,1.355196,0.202173,0.767846,1.290459,0.065722,LEFT_STEP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,9.812842,0.806147,0.440325,0.999648,0.816990,0.409700,0.999533,0.820650,0.409231,0.999476,...,0.850678,1.767658,0.008447,0.908015,1.764039,0.007685,0.839566,1.795935,0.009521,LEFT_STEP
305,9.812895,0.807728,0.437436,0.999546,0.818249,0.406794,0.999487,0.822152,0.406156,0.999401,...,0.863129,1.728501,0.007663,0.922270,1.748652,0.006942,0.850728,1.773876,0.008603,LEFT_STEP
306,9.911952,0.807728,0.437436,0.999546,0.818249,0.406794,0.999487,0.822152,0.406156,0.999401,...,0.863129,1.728501,0.007663,0.922270,1.748652,0.006942,0.850728,1.773876,0.008603,LEFT_STEP
307,9.912017,0.807728,0.437436,0.999546,0.818249,0.406794,0.999487,0.822152,0.406156,0.999401,...,0.863129,1.728501,0.007663,0.922270,1.748652,0.006942,0.850728,1.773876,0.008603,LEFT_STEP


In [41]:
# Conversion de 'action' en catégorie et codage
df_['action'] = df_['action'].astype('category')
action_code = pd.DataFrame(df_['action'].cat.codes, columns=['action_code'])
df_ = pd.concat([df_, action_code], axis=1)

# Affichage des catégories et des codes associés
categories = dict(enumerate(df_['action'].cat.categories))
print("Mapping des codes :", categories)

print("\nDataFrame transformé :")
df_.drop(['action'], axis=1, inplace=True)
df_

Mapping des codes : {0: 'LEFT_STEP'}

DataFrame transformé :


,relative_time,NOSE_x,NOSE_y,NOSE_p,LEFT_EYE_INNER_x,LEFT_EYE_INNER_y,LEFT_EYE_INNER_p,LEFT_EYE_x,LEFT_EYE_y,LEFT_EYE_p,...,RIGHT_HEEL_x,RIGHT_HEEL_y,RIGHT_HEEL_p,LEFT_FOOT_INDEX_x,LEFT_FOOT_INDEX_y,LEFT_FOOT_INDEX_p,RIGHT_FOOT_INDEX_x,RIGHT_FOOT_INDEX_y,RIGHT_FOOT_INDEX_p,action_code
0,0.000003,0.908151,0.377057,0.991300,0.907933,0.337525,0.989030,0.910916,0.334304,0.989894,...,0.917690,1.573847,0.106442,0.855478,1.598760,0.189950,0.826515,1.628760,0.062048,0
1,0.000185,0.908151,0.377057,0.991300,0.907933,0.337525,0.989030,0.910916,0.334304,0.989894,...,0.917690,1.573847,0.106442,0.855478,1.598760,0.189950,0.826515,1.628760,0.062048,0
2,0.000274,0.878059,0.377101,0.992086,0.881818,0.339911,0.990027,0.885482,0.337215,0.990794,...,0.851452,1.302500,0.113600,0.757849,1.355196,0.202173,0.767846,1.290459,0.065722,0
3,0.056491,0.878059,0.377101,0.992086,0.881818,0.339911,0.990027,0.885482,0.337215,0.990794,...,0.851452,1.302500,0.113600,0.757849,1.355196,0.202173,0.767846,1.290459,0.065722,0
4,0.056658,0.878059,0.377101,0.992086,0.881818,0.339911,0.990027,0.885482,0.337215,0.990794,...,0.851452,1.302500,0.113600,0.757849,1.355196,0.202173,0.767846,1.290459,0.065722,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,9.812842,0.806147,0.440325,0.999648,0.816990,0.409700,0.999533,0.820650,0.409231,0.999476,...,0.850678,1.767658,0.008447,0.908015,1.764039,0.007685,0.839566,1.795935,0.009521,0
305,9.812895,0.807728,0.437436,0.999546,0.818249,0.406794,0.999487,0.822152,0.406156,0.999401,...,0.863129,1.728501,0.007663,0.922270,1.748652,0.006942,0.850728,1.773876,0.008603,0
306,9.911952,0.807728,0.437436,0.999546,0.818249,0.406794,0.999487,0.822152,0.406156,0.999401,...,0.863129,1.728501,0.007663,0.922270,1.748652,0.006942,0.850728,1.773876,0.008603,0
307,9.912017,0.807728,0.437436,0.999546,0.818249,0.406794,0.999487,0.822152,0.406156,0.999401,...,0.863129,1.728501,0.007663,0.922270,1.748652,0.006942,0.850728,1.773876,0.008603,0


In [47]:
df_.to_csv('../data/2024-11-22-11-30-11_LEFT_STEP_transformed.csv', index=False)

In [66]:
N_POSE_FEATURES = 33*3
N_MIN_POSE = 39*2
ACTION_DURATION = 39*2

class DataCollatorForPoseMoveDetection(Dataset):
    
    def __init__(self,
                df_input,
                n_min_pose = N_MIN_POSE,
                action_duration = ACTION_DURATION):

        self.df_input = df_input
        self.n_min_pose = n_min_pose
        self.action_duration = action_duration
        
    def __len__(self):
        
        return len(self.df_input) - 1

    def __getitem__(self, idx):

        """
        idx is the idx of an element in the dataset, a number between 0 and len(dataset)
        """

        if idx<=self.n_min_pose:
            idx = self.n_min_pose + torch.randint(0, len(self.df_input)-self.n_min_pose, (1,)).item()

        input = self.df_input.iloc[idx-self.action_duration:idx, :]
        positions = torch.tensor(input['relative_time'].values) # time positions
        pose_features = torch.tensor(input.iloc[:, 1:-1].values) # pose features, each point with (x,y,p)
        labels = torch.tensor(input['action_code'].values[-1]) # action of the last frame
        attention_mask = torch.ones(self.action_duration) # no masking at this point
        
        sample = {"labels": labels,
                  "pose_features": pose_features, 
                  "positions": positions, 
                  "attention_mask": attention_mask}

        return sample


In [67]:
import json
import glob
from torch.utils.data import ConcatDataset, random_split

In [69]:
config = json.load(open("../config/movedetect.json", "r"))

#----------------- Load the data -----------------#
datafile, batch_size = config["datafile"], config["batch_size"]
dev_ratio, seed = config["dev_ratio"], config["seed"]

files = glob.glob("../" + datafile+'/*.csv')

all_datasets = []
for file in files:
    df_input = pd.read_csv(file)
    all_datasets.append(DataCollatorForPoseMoveDetection(df_input))

my_dataset = ConcatDataset(all_datasets[1:])
# split the dataset 80/20
train_dataset, eval_dataset = random_split(my_dataset, [int((1-dev_ratio)*len(my_dataset)), len(my_dataset) - int((1-dev_ratio)*len(my_dataset))], generator=torch.Generator().manual_seed(seed))

In [ ]:
eval_labels_histogram = []
train_labels_histogram = []

for sample in tqdm(eval_dataset):
    labels = sample['labels']
    eval_labels_histogram.append(labels.item())

for sample in tqdm(train_dataset):
    labels = sample['labels']
    train_labels_histogram.append(labels.item())

In [ ]:
actions2labels = {'ACTIVATE':1, 
 'CROUCH':2, 
 'IDLE':0, # nothing
 'INFO_DISPLAYED':0, 
 'JUMP':3,
 'LEFT_STEP':0, 
 'RIGHT_STEP':0, 
 'SHAKE': 4}


In [75]:
print(np.unique(eval_labels_histogram))
print(np.unique(eval_labels_histogram, return_counts=True)[1]/len(eval_labels_histogram))
print(np.unique(train_labels_histogram, return_counts=True)[1]/len(train_labels_histogram))

[0 1 2 3 4]
[0.49616781 0.16216216 0.15288423 0.08995563 0.09883017]
[0.49028124 0.16852973 0.15742257 0.08942684 0.09433962]
